In [ ]:
import argparse
import os
import numpy as np
import trimesh
import sys
from matplotlib import pyplot as plt
from PIL import Image

sys.path.append(os.path.dirname(os.path.abspath('')))
import utils.FPHA_utils as FPHA
# Loading utilities

In [ ]:
root = FPHA.DIR
train_file_name, test_file_name, _, _ = FPHA.get_train_test_pairs('color', root)
train_file_name = [file.split('/') for file in train_file_name]
test_file_name = [file.split('/') for file in test_file_name]


In [ ]:
# file_name = train_file_name
# idx = 34110                 
# subject = file_name[idx][0]
# action_name = file_name[idx][1]
# seq_idx = file_name[idx][2]
# frame_idx = int(file_name[idx][-1][-9:-5])
# obj = None

In [ ]:
# root = FPHA.DIR
# subject = 'Subject_1'
# action_name = 'open_liquid_soap'
# seq_idx = '1'
# frame_idx = 0
# obj = 'liquid_soap'
# # obj = None
path = 'Subject_2/open_letter/2/color/color_0470.jpeg'.split('/')
subject = path[0]
action_name = path[1]
seq_idx = path[2]
frame_idx = int(path[4][6:10])
obj = None

In [ ]:
sample = {
    'subject': subject,
    'action_name': action_name,
    'seq_idx': seq_idx,
    'frame_idx': frame_idx,
    'object': obj
}

print('Loading sample {}'.format(sample))

cam_extr = np.array(FPHA.CAM_EXTR)
cam_intr_color = np.array(FPHA.CAM_INTR_COLOR)
cam_intr_depth = np.array(FPHA.CAM_INTR_DEPTH)

skeleton_root = os.path.join(root, 'Hand_pose_annotation_v1')
obj_root = os.path.join(root, 'Object_models')
obj_trans_root = os.path.join(root, 'Object_6D_pose_annotation_v1')
skel = FPHA.get_skeleton(sample, skeleton_root)[FPHA.REORDER_IDX]

if obj is not None:
    # Load object mesh
    object_infos = FPHA.load_objects(obj_root)

    # Load object transform
    obj_trans = FPHA.get_obj_transform(sample, obj_trans_root)

    # Get object vertices
    verts = object_infos[sample['object']]['verts'] * 1000

    # Apply transform to object
    hom_verts = np.concatenate(
        [verts, np.ones([verts.shape[0], 1])], axis=1)
    verts_trans = obj_trans.dot(hom_verts.T).T

    # Apply camera extrinsic to objec
    verts_camcoords = cam_extr.dot(
        verts_trans.transpose()).transpose()[:, :3]
    # Project and object skeleton using camera intrinsics
    verts_hom2d = np.array(cam_intr_color).dot(
        verts_camcoords.transpose()).transpose()
    verts_proj = (verts_hom2d / verts_hom2d[:, 2:])[:, :2]

# Apply camera extrinsic to hand skeleton
skel_hom = np.concatenate([skel, np.ones([skel.shape[0], 1])], 1)
skel_camcoords = cam_extr.dot(
    skel_hom.transpose()).transpose()[:, :3].astype(np.float32)

skel_hom2d = np.array(cam_intr_color).dot(skel_camcoords.transpose()).transpose()
skel_proj = (skel_hom2d / skel_hom2d[:, 2:])[:, :2]

# Plot everything
# Load image and display
img_path = os.path.join(root, 'Video_files', sample['subject'],
                        sample['action_name'], sample['seq_idx'], 'color',
                        'color_{:04d}.jpeg'.format(sample['frame_idx']))
print('Loading image from {}'.format(img_path))
img = Image.open(img_path)
img = np.asarray(img)
print('Width:', img.shape[1], 'Height:', img.shape[0])
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(221)
ax.imshow(img)
FPHA.visualize_joints_2d(ax, skel_proj, joint_idxs=False)

if obj is not None:
    ax.scatter(verts_proj[:, 0], 
               verts_proj[:, 1], 
               alpha=0.01, 
               c='r')
    
for proj_idx, (proj_1, proj_2) in enumerate([[0, 1], [1, 2], [0, 2]]):
    ax = fig.add_subplot(2, 2, 2 + proj_idx)
    if proj_idx == 0:
        # Invert y axes to align with image in camera projection
        ax.invert_yaxis()
    ax.set_aspect('equal')
    if obj is not None:
        ax.scatter(verts_camcoords[:, proj_1], 
                   verts_camcoords[:, proj_2], s=1)
    FPHA.visualize_joints_2d(ax,
                        np.stack(
                            [skel_camcoords[:, proj_1], 
                             skel_camcoords[:, proj_2]],
                            axis=1),
                        joint_idxs=False)
        


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,15))
ax[0].imshow(img)
FPHA.visualize_joints_2d(ax[0], skel_proj, joint_idxs=False)

skel_hom2d = np.array(cam_intr_depth).dot(skel.transpose()).transpose()
skel_proj = (skel_hom2d / skel_hom2d[:, 2:])[:, :2]

# Plot everything
# Load image and display
# ax = fig.add_subplot(221)
img_path = os.path.join(root, 'Video_files', sample['subject'],
                        sample['action_name'], sample['seq_idx'], 'depth',
                        'depth_{:04d}.png'.format(sample['frame_idx']))
print('Loading image from {}'.format(img_path))
img = Image.open(img_path)
# ax.imshow(img)

ax[1].imshow(img)
FPHA.visualize_joints_2d(ax[1], skel_proj, joint_idxs=False)